<a href="https://colab.research.google.com/github/Zenith1618/LLM/blob/main/Ad_generation_from_Product_Description_Using_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00


In [2]:
import torch
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from datasets import Dataset

In [3]:
model_name = "gpt2-large" #or just gpt2
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = TFGPT2LMHeadModel.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [5]:
def generate_advertisement(product_description, max_length=150):
    input_text = "Product: " + product_description + "\nAdvertisement:"

    # Encode input text into ids- tokenization
    input_ids = tokenizer.encode(input_text, return_tensors="tf")

    # Generate text
    output = model.generate(input_ids, max_length=max_length)

    # decode the ids back into text
    generated_ads = []
    for sample in output:
        generated_ad = tokenizer.decode(sample, skip_special_tokens=True)
        generated_ads.append(generated_ad)

    return generated_ads

In [6]:
product_description = "Introducing our latest smartphone model, with a powerful processor and stunning camera features."

# Generate advertisements
generated_ads = generate_advertisement(product_description, max_length=150)

In [7]:
generated_ads

['Product: Introducing our latest smartphone model, with a powerful processor and stunning camera features.\nAdvertisement:\nThe new model is the first to feature a 5.5-inch display, a Qualcomm Snapdragon 810 processor, 4GB of RAM, and a 13MP rear camera with a f/2.0 aperture. The phone also features a fingerprint sensor, a 3,000mAh battery, and a 3,000mAh removable battery. The phone will be available in two colors: black and white.\nThe phone will be available in China starting on September 1st, and will be priced at 2,499 yuan (about $400).']

## Using Greedy Approach

With Greedy search, the word with the highest probability is predicted as the next word:

$w_t=argmax_wP(w|w_1:_t-_1)$

Beam search is essentially Greedy Search but the model tracks and keeps num_beams of hypotheses at each time step, so the model is able to compare alternative paths as it generates text. We can also include a n-gram penalty by setting no_repeat_ngram_size = 3 which ensures that no 3-grams appear thrice

In [8]:
def generate_advertisement_greedy(product_description):
    input_text = "Product: " + product_description + "\nAdvertisement:"

    # Encode input text- use number of beams, ngram size
    input_ids = tokenizer.encode(input_text, num_beams = 7,no_repeat_ngram_size=3,num_return_sequences=5,early_stopping = True,return_tensors="tf")

    # Generate text
    output = model.generate(input_ids, max_length=150)

    # decode the ids back into text
    generated_ads = []
    for sample in output:
        generated_ad = tokenizer.decode(sample, skip_special_tokens=True)
        generated_ads.append(generated_ad)

    return generated_ads

In [9]:
generated_ads_greedy = generate_advertisement_greedy(product_description)

Keyword arguments {'num_beams': 7, 'no_repeat_ngram_size': 3, 'num_return_sequences': 5, 'early_stopping': True} not recognized.


In [10]:
generated_ads_greedy

['Product: Introducing our latest smartphone model, with a powerful processor and stunning camera features.\nAdvertisement:\nThe new model is the first to feature a 5.5-inch display, a Qualcomm Snapdragon 810 processor, 4GB of RAM, and a 13MP rear camera with a f/2.0 aperture. The phone also features a fingerprint sensor, a 3,000mAh battery, and a 3,000mAh removable battery. The phone will be available in two colors: black and white.\nThe phone will be available in China starting on September 1st, and will be priced at 2,499 yuan (about $400).']